In [1]:
import numpy as np
import pandas as pd

In [3]:
# Change the k parameter only (sketch matrix size -- determine how much compression)
# for example, if k=85, epsilon will be 5; if k=58, epsilon will be 2.5; if k=36, epsilon will be 1; if k=26, epsilon will be 0.5  
import math
import numpy as np
# alpha is the bound, sigma is the variance
alpha = 20.0
sigma = (alpha*alpha)/4.0

n = 144.0    # (1 day information)
t = 200.0
k = 36.0     # compression -- change it to get different epsilon value
s = k

B = 1 / (1/2 - ((alpha*alpha*k*(k-1)) / (alpha*alpha*(n-2))) * (1+math.log(n-k)) )
print("B value should be positive and greater than ", B)

B value should be positive and greater than  -0.020032488333575224


In [4]:
B = 0.0001
epsilon = t * math.log(1 + ((B*alpha*alpha*k*(k-1)) / (alpha*alpha*(n-2))) * (1+math.log(n-k)))
epsilon

1.005844663563975

In [5]:
df = pd.read_csv('../data/Residential-Profiles.csv')
df = df.drop(columns='Time')

In [6]:
months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
indices = list(np.cumsum((np.array(months) * 144)))
indices = [0] + indices
print(indices)

[0, 4464, 8496, 12960, 17280, 21744, 26064, 30528, 34992, 39312, 43776, 48096, 52560]


In [7]:
# streaming implementation
def countSketchStreaming(matrixA, s):
    s = int(s)
    m,n = matrixA.shape
    matrixC = np.zeros([m, s])
    hashedIndices = np.random.choice(s, n, replace=True)
    randSigns = np.random.choice(2, n, replace = True) * 2 - 1
    for j in range(n):
        a = matrixA[:, j]
        h = hashedIndices[j]
        g = randSigns[j]
        matrixC[:,h] += g * a
    return matrixC

In [8]:
count = 0
total_MAE = 0
for house in houses:
    for i in range(len(indices)-1):
        inputMatrix = np.transpose(np.array(df.loc[indices[i]:(indices[i+1]-1),house]))
        inputMatrix = np.reshape(inputMatrix, (months[i],-1))
        matrixC = countSketchStreaming(inputMatrix, s)


        rowNormsC = np.sqrt(np.sum(np.square(matrixC), 1))
        distributionC = rowNormsC/sum(rowNormsC)
        rowNormsA = np.sum(inputMatrix, axis=1)
        distributionA = rowNormsA/sum(rowNormsA)

        MAE = sum(abs(distributionC-distributionA))/len(distributionA)
        total_MAE += MAE
        count = count + 1

MAE = total_MAE/count
print("For epsilon ", epsilon, ", Mean Absolute Error (MAE) is ", MAE)

For epsilon  1.005844663563975 , Mean Absolute Error (MAE) is  0.0026651422889349703
